# Resume Parsing

In [1]:
import fitz  # PyMuPDF
import spacy
from spacy.matcher import PhraseMatcher

1. Using PhraseMatcher

In [2]:
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error opening or reading PDF: {e}")
        return None

def extract_skills_and_points(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)

    skills = []
    key_points = []

    # Define some common skills to search for
    skill_phrases = [
        "machine learning", "deep learning", "data analysis", "python", "java",
        "project management", "communication", "teamwork", "sql", "excel"
    ]
    matcher = PhraseMatcher(nlp.vocab)
    patterns = [nlp(skill) for skill in skill_phrases]
    matcher.add("SKILLS", patterns)

    # Use matcher to find skills
    matches = matcher(doc)
    for match_id, start, end in matches:
        skills.append(doc[start:end].text)

    # Extract key points
    for token in doc:
        if token.pos_ == 'VERB' and token.dep_ == 'ROOT':
            key_points.append(token.text)

    return skills, key_points

if __name__ == "__main__":
    # Replace with your actual path to the resume PDF file
    pdf_path = '/content/resume.pdf'

    resume_text = extract_text_from_pdf(pdf_path)

    if resume_text:
        skills, key_points = extract_skills_and_points(resume_text)

        print("Skills:")
        for skill in skills:
            print(skill)

        print("\nKey Points:")
        for point in key_points:
            print(point)

Error opening or reading PDF: no such file: '/content/resume.pdf'


2. Using BST

In [3]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_phrase = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, phrase):
        node = self.root
        for char in phrase:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_phrase = True

    def search(self, text):
        node = self.root
        matched_phrases = []
        i = 0
        while i < len(text):
            char = text[i]
            if char in node.children:
                temp_node = node
                temp_phrase = ''
                while i < len(text) and char in temp_node.children:
                    temp_phrase += char
                    temp_node = temp_node.children[char]
                    if temp_node.is_end_of_phrase:
                        matched_phrases.append(temp_phrase)
                    i += 1
                    if i < len(text):
                        char = text[i]
                i -= len(temp_phrase) - 1
            i += 1
        return matched_phrases

In [4]:
import pandas as pd
import re

df = pd.read_csv("Data/job_descriptions.csv")

irrelevant_words = [
    "knowledge", "and", "skills", "control", "including"
    "codes", "regulations", "cost", "monitor", "timelines",
    "assist", "in", "budgeting"
]

# Function to clean and extract skills
pattern = re.compile(r'\b(?:' + '|'.join(irrelevant_words) + r')\b', re.IGNORECASE)

# Function to clean and extract skills
def clean_skills(skills_string):
    # Remove brackets and their contents
    skills_string = re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', skills_string)
    
    # Clean the skills string by removing irrelevant words
    cleaned_skills = pattern.sub('', skills_string)
    
    # Split the cleaned string into a list of skills
    skill_list = re.split(r'\s*,\s*|\s+', cleaned_skills.strip())
    
    # Remove any empty strings from the list
    skill_list = [skill for skill in skill_list if skill]
    
    return skill_list

# Apply the function to the 'skills' column and generate the list of cleaned skills
df['cleaned_skills'] = df['skills'].apply(clean_skills)

# Print the DataFrame to see the results
print(df[['skills', 'cleaned_skills']])

                                                   skills  \
0       Construction project management Building codes...   
1       Procurement processes Purchase order managemen...   
2       Proficiency in HTML, CSS, and JavaScript Exper...   
3       Marketing analytics Data analysis Data visuali...   
4       Accounting principles Financial reporting Team...   
...                                                   ...   
698401  Pediatric specialization Advanced pediatric ca...   
698402  Data analysis Database querying and reporting ...   
698403  User-centered design principles UX/UI design t...   
698404  Content creation (e.g., writing, editing, grap...   
698405  Content creation (e.g., writing, editing, grap...   

                                           cleaned_skills  
0       [Construction, project, management, Building, ...  
1       [Procurement, processes, Purchase, order, mana...  
2       [Proficiency, HTML, CSS, JavaScript, Experienc...  
3       [Marketing, analyti

In [5]:

# Usage of Trie in the main script
def extract_skills_and_points(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)

    skills = []
    key_points = []

    # Define some common skills to search for
    skill_phrases = [
        "machine learning", "deep learning", "data analysis", "python", "java",
        "project management", "communication", "teamwork", "sql", "excel"
    ]

    # Build the trie
    trie = Trie()
    for phrase in skill_phrases:
        trie.insert(phrase.lower())

    # Use trie to find skills
    lower_text = text.lower()
    skills = trie.search(lower_text)

    # Extract key points
    for token in doc:
        if token.pos_ == 'VERB' and token.dep_ == 'ROOT':
            key_points.append(token.text)

    return skills, key_points

if __name__ == "__main__":
    # Replace with your actual path to the resume PDF file
    pdf_path = '/content/resume.pdf'

    resume_text = extract_text_from_pdf(pdf_path)

    if resume_text:
        skills, key_points = extract_skills_and_points(resume_text)

        print("Skills:")
        for skill in skills:
            print(skill)

        print("\nKey Points:")
        for point in key_points:
            print(point)


Error opening or reading PDF: no such file: '/content/resume.pdf'


# Re-Sampling

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [7]:
dataset = pd.read_csv("Data/data_after_feature_eng.csv")

In [8]:
pd.DataFrame(dataset["Job Title"].value_counts())

,count
Job Title,
UX/UI Designer,20907
Digital Marketing Specialist,11970
Software Engineer,11912
Network Engineer,10500
Software Tester,9087
...,...
Procurement Coordinator,1468
Personal Assistant,1461
Financial Planner,1455


In [9]:
X = dataset.drop(columns=["Job Title"])
y = dataset["Job Title"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
from imblearn.over_sampling import SMOTE

In [12]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
from collections import Counter

In [14]:
print(f"Original dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_resampled)}")

Original dataset shape: Counter({'UX/UI Designer': 18793, 'Digital Marketing Specialist': 10735, 'Software Engineer': 10692, 'Network Engineer': 9467, 'Software Tester': 8206, 'Executive Assistant': 8113, 'Procurement Manager': 8038, 'Financial Advisor': 8005, 'Social Media Manager': 6910, 'Sales Representative': 6857, 'Network Administrator': 6819, 'Data Analyst': 6795, 'Purchasing Agent': 6769, 'Administrative Assistant': 6751, 'Event Planner': 6714, 'Systems Administrator': 6700, 'Procurement Specialist': 6694, 'Customer Support Specialist': 6660, 'HR Coordinator': 6629, 'Customer Success Manager': 5519, 'UI Developer': 5470, 'Legal Assistant': 5470, 'Marketing Analyst': 5429, 'Landscape Architect': 5420, 'Graphic Designer': 5414, 'Content Writer': 5408, 'Supply Chain Manager': 5400, 'Account Manager': 5394, 'Project Manager': 5391, 'Marketing Manager': 5391, 'Investment Banker': 5386, 'Event Coordinator': 5378, 'Operations Manager': 5375, 'Litigation Attorney': 5355, 'Research Anal

In [15]:
X = X_train_resampled.copy() 
y = y_train_resampled.copy()

In [16]:
X.shape, y.shape

((2762571, 67), (2762571,))

In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [18]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

In [20]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2486313, 67), (276258, 67), (2486313,), (276258,))

In [21]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [22]:
y_train = np.array(y_train)
y_test = np.array(y_test)
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2486313, 67), (276258, 67), (2486313, 147), (276258, 147))

In [24]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [25]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Starting learning rate
    decay_steps=100,            # Number of steps before applying decay
    decay_rate=0.96,             # Decay rate
    staircase=True               # Whether to apply decay at discrete steps
)

optimizer = Adam(learning_rate=lr_schedule)

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input

class MyModel(Model):
    def __init__(self, input_dim, num_classes):
        super(MyModel, self).__init__()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(1024, activation='relu')
        self.dense3 = Dense(1024, activation='relu')
        self.dense4 = Dense(1024, activation='relu')
        self.dense5 = Dense(2056, activation='relu')
        self.dense6 = Dense(2056, activation='relu')
        self.output_layer = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.output_layer(x)

# Define the input dimension and number of classes
input_dim = 67  # Example input dimension

# Create an instance of the model
model = MyModel(input_dim = input_dim, num_classes = num_classes)

# Build the model by providing an input shape
model.build((None, input_dim))

# Compile the model
model.compile(optimizer=optimizer , loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()




Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  8704      
                                                                 
 dense_1 (Dense)             multiple                  132096    
                                                                 
 dense_2 (Dense)             multiple                  1049600   
                                                                 
 dense_3 (Dense)             multiple                  1049600   
                                                                 
 dense_4 (Dense)             multiple                  2107400   
                                                                 
 dense_5 (Dense)             multiple                  4229192   
                                                                 
 dense_6 (Dense)             multiple                  3

In [27]:
model.fit(X_train, y_train, epochs = 2, validation_split=0.2, batch_size = 2048)

Epoch 1/2

972/972 [==============================] - 190s 194ms/step - loss: 4430.4092 - accuracy: 0.0068 - val_loss: 4.9909 - val_accuracy: 0.0066
Epoch 2/2
972/972 [==============================] - 178s 184ms/step - loss: 4.9909 - accuracy: 0.0068 - val_loss: 4.9908 - val_accuracy: 0.0067


# Job Recommendation system

In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [29]:
class JobRecommendationSystem:
    def __init__(self, data, cat_features, num_features, target_var):
        self.data = data
        self.cat_features = cat_features
        self.num_features = num_features
        self.target_var = target_var
        self.encoder = OneHotEncoder(sparse = False)
        self.scaler = StandardScaler()
        self.job_title_encoder = OneHotEncoder(sparse = False)
        
    def preprocess_data(self):
        # Encode categorical features
        encoded_features = self.encoder.fit_transform(self.data[self.cat_features])
        encoded_df = pd.DataFrame(encoded_features, columns = self.encoder.get_feature_names_out())
        
        # Normalize numerical features
        numerical_features = self.scaler.fit_transform(self.data[self.num_features])
        numerical_df = pd.DataFrame(numerical_features, columns = self.num_features)
        
        # Flatten the skills list and encode
        # The skills column must be list of list
        all_skills = list(set(skill for sublist in self.data["skills"] for skill in sublist))
        skills_df = pd.DataFrame([[skill in sublist for skill in all_skills] for sublist in self.data["skills"]], columns = all_skills)
        
        # Combine all features
        self.X = pd.concat([encoded_df, numerical_df, skills_df], axis = 1)
        
        # Encode the Job Title
        self.y = self.job_title_encoder.fit_transform(self.data[self.target_var])
        
        # Split the data into training and test sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.2, random_state = 42)

    def build_model(self):
        # Define the neural network model using the Functional API
        input_layer = Input(shape = (self.X_train.shape[1],))
        dense_layer_1 = Dense(64, activation = "relu")(input_layer)
        dense_layer_2 = Dense(32, activation = "relu")(dense_layer_1)
        output_layer = Dense(self.y.shape[1], activation = "softmax")(dense_layer_2)
        
        self.model = Model(inputs = input_layer, outputs = output_layer)
        self.model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    
    def train_model(self, epochs = 10, batch_size = 10):
        self.model.fit(self.X_train, self.y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2)
    
    def evaluate_model(self):
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        print(f"Test Accuracy: {accuracy}")
    
    def make_predictions(self):
        predictions = self.model.predict(self.X_test)
        predicted_classes = self.job_title_encoder.inverse_transform(predictions)
        return predicted_classes

# Job Recommendation

In [30]:
dataset = pd.read_csv("Data/data_after_feature_eng.csv")

In [31]:
X = dataset.drop(columns=["Job Title"])
y = dataset["Job Title"]

In [32]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [33]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

In [36]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((626410, 67), (69602, 67), (626410,), (69602,))

In [37]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [38]:
y_train = np.array(y_train)
y_test = np.array(y_test)
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [39]:
from tensorflow.keras.layers import Layer

In [40]:
class QuadDense(Layer):

  def __init__(self, units = 32, activation = None):
    super(QuadDense, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, input_shape):
    a_init = tf.random_normal_initializer()
    b_init = tf.random_normal_initializer()
    c_init = tf.zeros_initializer()

    self.a = tf.Variable(initial_value = a_init(shape = (input_shape[-1], self.units),
                                                dtype = "float32"),
                         trainable = True,
                         name = "kernel")

    self.b = tf.Variable(initial_value = b_init(shape = (input_shape[-1], self.units),
                                                dtype = "float32"),
                         trainable = True,
                         name = "kernel")

    self.c = tf.Variable(initial_value = c_init(shape = (self.units, ),
                                                dtype = "float32"),
                         trainable = True,
                         name = "bias")

  def call(self, inputs):
    result = tf.matmul(tf.math.square(inputs), self.a) + tf.matmul(inputs, self.b) + self.c

    return self.activation(result)

In [41]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [42]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Starting learning rate
    decay_steps=1000,            # Number of steps before applying decay
    decay_rate=0.96,             # Decay rate
    staircase=True               # Whether to apply decay at discrete steps
)

optimizer = Adam(learning_rate=lr_schedule)

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input

class MyModel(Model):
    def __init__(self, input_dim, num_classes):
        super(MyModel, self).__init__()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(512, activation='relu')
        self.dense4 = QuadDense(2056, activation='relu')
        self.output_layer = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.output_layer(x)

# Define the input dimension and number of classes
input_dim = 67  # Example input dimension

# Create an instance of the model
model = MyModel(input_dim = input_dim, num_classes = num_classes)

# Build the model by providing an input shape
model.build((None, input_dim))

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             multiple                  8704      
                                                                 
 dense_8 (Dense)             multiple                  33024     
                                                                 
 dense_9 (Dense)             multiple                  131584    
                                                                 
 quad_dense (QuadDense)      multiple                  2107400   
                                                                 
 dense_10 (Dense)            multiple                  302379    
                                                                 
Total params: 2583091 (9.85 MB)
Trainable params: 2583091 (9.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
model.fit(X_train, y_train, epochs = 5, validation_split=0.2, batch_size = 128)

Epoch 1/5
3916/3916 [==============================] - 50s 13ms/step - loss: 4.8856 - accuracy: 0.0300 - val_loss: 4.8853 - val_accuracy: 0.0299
Epoch 2/5
3916/3916 [==============================] - 49s 12ms/step - loss: 4.8848 - accuracy: 0.0300 - val_loss: 4.8849 - val_accuracy: 0.0299
Epoch 3/5
3916/3916 [==============================] - 48s 12ms/step - loss: 4.8844 - accuracy: 0.0300 - val_loss: 4.8849 - val_accuracy: 0.0299
Epoch 4/5
3916/3916 [==============================] - 48s 12ms/step - loss: 4.8842 - accuracy: 0.0300 - val_loss: 4.8844 - val_accuracy: 0.0299
Epoch 5/5
3916/3916 [==============================] - 48s 12ms/step - loss: 4.8841 - accuracy: 0.0300 - val_loss: 4.8843 - val_accuracy: 0.0299


In [46]:
from tensorflow.keras.optimizers import Adam

# Create an Adam optimizer with a custom learning rate
optimizer = Adam(learning_rate=0.0001)

In [47]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Conv1D, Flatten, BatchNormalization, Dropout, Attention, Add
from tensorflow.keras.models import Model

# Define a custom layer QuadDense
class QuadDense(Layer):
    def __init__(self, units=32, activation=None):
        super(QuadDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        a_init = tf.random_normal_initializer()
        b_init = tf.random_normal_initializer()
        c_init = tf.zeros_initializer()

        self.a = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer=a_init,
                                 trainable=True,
                                 name="a")

        self.b = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer=b_init,
                                 trainable=True,
                                 name="b")

        self.c = self.add_weight(shape=(self.units,),
                                 initializer=c_init,
                                 trainable=True,
                                 name="c")

    def call(self, inputs):
        result = tf.matmul(tf.math.square(inputs), self.a) + tf.matmul(inputs, self.b) + self.c
        return self.activation(result)

# Define a model that uses various layers for better performance
class MyModel(Model):
    def __init__(self, input_dim, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = Conv1D(64, kernel_size=3, activation='relu', padding='same')
        self.conv2 = Conv1D(128, kernel_size=3, activation='relu', padding='same')
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.batch_norm1 = BatchNormalization()
        self.dropout1 = Dropout(0.2)
        self.dense2 = Dense(1024, activation='relu')
        self.batch_norm2 = BatchNormalization()
        self.dropout2 = Dropout(0.2)
        
        # Adding attention layers
        self.attention1 = Attention()
        self.attention2 = Attention()
        
        self.dense3 = QuadDense(1024, activation='relu')
        self.dense4 = QuadDense(1024, activation='relu')
        self.dense5 = Dense(2056, activation='relu')
        self.batch_norm3 = BatchNormalization()
        self.dropout3 = Dropout(0.2)
        self.dense6 = Dense(2056, activation='relu')
        self.output_layer = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.conv1(tf.expand_dims(inputs, axis=-1))  # Expanding dims to use Conv1D
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        
        # Applying attention
        query = tf.expand_dims(x, axis=1)  # Adding sequence dimension for Attention layer
        attention_output1 = self.attention1([query, query])
        x = Add()([x, tf.squeeze(attention_output1, axis=1)])
        
        attention_output2 = self.attention2([query, query])
        x = Add()([x, tf.squeeze(attention_output2, axis=1)])
        
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense6(x)
        return self.output_layer(x)

# Define the input dimension and number of classes
input_dim = 67  # Example input dimension

# Create an instance of the model
model = MyModel(input_dim=input_dim, num_classes=num_classes)

# Build the model by providing an input shape
model.build((None, input_dim))

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "my_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  256       
                                                                 
 conv1d_1 (Conv1D)           multiple                  24704     
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense_11 (Dense)            multiple                  1097856   
                                                                 
 batch_normalization (Batch  multiple                  512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           multiple                  0         
                                                        

In [56]:
model.fit(X_train, y_train, epochs = 5, validation_split=0.2, batch_size = 2048)

Epoch 1/5
245/245 [==============================] - 111s 452ms/step - loss: 4.8956 - accuracy: 0.0298 - val_loss: 16.6723 - val_accuracy: 0.0143
Epoch 2/5
245/245 [==============================] - 110s 447ms/step - loss: 4.9035 - accuracy: 0.0295 - val_loss: 4.9112 - val_accuracy: 0.0299
Epoch 3/5
245/245 [==============================] - 109s 445ms/step - loss: 4.8940 - accuracy: 0.0299 - val_loss: 4.8905 - val_accuracy: 0.0297
Epoch 4/5
245/245 [==============================] - 107s 437ms/step - loss: 4.8915 - accuracy: 0.0299 - val_loss: 4.8904 - val_accuracy: 0.0297
Epoch 5/5
245/245 [==============================] - 107s 437ms/step - loss: 4.8902 - accuracy: 0.0299 - val_loss: 4.8894 - val_accuracy: 0.0298


In [57]:
predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

2176/2176 [==============================] - 12s 6ms/step


In [58]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

2176/2176 [==============================] - 13s 6ms/step - loss: 4.8867 - accuracy: 0.0303
Test Loss: 4.886653900146484
Test Accuracy: 0.030329588800668716


In [51]:
y_test_classes = np.argmax(y_test, axis=1)

# Assuming X_test is a 2D array where each row is a sample
# Convert X_test to a DataFrame if it's not already
X_test_df = pd.DataFrame(X_test)

# Create a DataFrame to hold original values, true labels, and predictions
results_df = pd.DataFrame({
    'True Labels': y_test_classes,
    'Predicted Labels': predicted_classes
})

print(results_df.head())  # Display the first few rows of the DataFrame


   True Labels  Predicted Labels
0           40               141
1           94               141
2           62               141
3           90               141
4           35               141


In [59]:
results_df["Predicted Labels"].unique()

array([141,   2, 126], dtype=int64)

In [60]:
import pickle

In [61]:
# Specify the filename where you want to save the model
filename = 'Data/model.pkl'

# Open a file in write-binary mode and use pickle.dump to save the model
with open(filename, 'wb') as file:
    pickle.dump(model, file)


#### Salary Prediction

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import keras

In [ ]:
class Linear_Model:
    def __init__(self, input_dim, hidden_layers, output_dim):
        """
        Parameters:
        input_dim (int): Number of the input features
        hidden_layers (list): list of the number of units in the hidden layer
        output_dim: nNumber of the output units 
        """
        self.model = Sequential()
        
        # Input layer
        self.model.add(Dense(hidden_layers[0], input_dim = input_dim, activation = "relu"))
        
        # Hidden layers
        for units in hidden_layers[1:]:
            self.model.add(Dense(units, activation = "relu"))
        
        # Output layer
        self.model.add(Dense(output_dim, activation = "linear"))
        
        # Compile the model
        self.model.compile(optimizer = "adam", loss = "mse", metrics = ["mse"])
    
    def train(self, X_train, y_train, epochs = 10, batch_size = 32):
        self.model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)
    
    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)
    
    def predict(self, X):
        return self.model.predict(X)